In [151]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [152]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "AVG-ICA"

In [153]:
# /datassd/WHOLEIMAGE_MAMIP/EXTRACTED_FEATURES
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [154]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031
0,CHUM-001,-0.006003,-0.005180,0.000247,-0.003564,0.002391,0.000657,0.001300,-0.004477,-0.001794,...,-0.005790,-0.006003,0.000588,-0.006003,-0.005886,0.004307,0.005273,-0.002276,-0.006003,-0.004833
1,CHUM-002,-0.006060,-0.005258,0.000122,-0.003352,0.001866,-0.000670,0.000099,-0.004938,-0.002726,...,-0.005696,-0.006027,-0.001268,-0.006060,-0.005873,0.005005,0.006572,-0.002019,-0.006060,-0.004483
2,CHUM-006,0.006663,0.003941,0.002593,0.006524,-0.001161,-0.000918,-0.005707,0.006610,0.004425,...,0.006584,0.006652,0.000761,0.006665,0.006503,-0.003080,-0.003952,0.001996,0.006665,0.004968
3,CHUM-007,0.006676,0.004293,0.001961,0.006474,-0.005855,0.000587,-0.004173,0.006583,0.004402,...,0.006676,0.006672,0.000425,0.006676,0.006676,-0.002387,-0.001664,0.003379,0.006676,0.006145
4,CHUM-008,-0.006672,-0.005507,-0.001959,-0.005280,0.001628,-0.000958,-0.000569,-0.005519,-0.003531,...,-0.006572,-0.006570,0.000808,-0.006672,-0.006657,0.003432,0.004565,-0.002438,-0.006672,-0.006045


# Up to here

Read Outcome csv

In [155]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [156]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [157]:
OD.columns[0]

'Patient_ID'

In [158]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [159]:
OD.columns[0]


'Patient_ID'

In [160]:
PET_outcome_data.columns[0]

'Patient_ID'

In [161]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031
0,CHUM-001,-0.006003,-0.005180,0.000247,-0.003564,0.002391,0.000657,0.001300,-0.004477,-0.001794,...,-0.005790,-0.006003,0.000588,-0.006003,-0.005886,0.004307,0.005273,-0.002276,-0.006003,-0.004833
1,CHUM-002,-0.006060,-0.005258,0.000122,-0.003352,0.001866,-0.000670,0.000099,-0.004938,-0.002726,...,-0.005696,-0.006027,-0.001268,-0.006060,-0.005873,0.005005,0.006572,-0.002019,-0.006060,-0.004483
2,CHUM-006,0.006663,0.003941,0.002593,0.006524,-0.001161,-0.000918,-0.005707,0.006610,0.004425,...,0.006584,0.006652,0.000761,0.006665,0.006503,-0.003080,-0.003952,0.001996,0.006665,0.004968
3,CHUM-007,0.006676,0.004293,0.001961,0.006474,-0.005855,0.000587,-0.004173,0.006583,0.004402,...,0.006676,0.006672,0.000425,0.006676,0.006676,-0.002387,-0.001664,0.003379,0.006676,0.006145
4,CHUM-008,-0.006672,-0.005507,-0.001959,-0.005280,0.001628,-0.000958,-0.000569,-0.005519,-0.003531,...,-0.006572,-0.006570,0.000808,-0.006672,-0.006657,0.003432,0.004565,-0.002438,-0.006672,-0.006045


In [162]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [163]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [164]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,4024,4025,4026,4027,4028,4029,4030,4031,Relapse,RFS
0,CHUM-001,-0.006003,-0.005180,0.000247,-0.003564,0.002391,0.000657,0.001300,-0.004477,-0.001794,...,0.000588,-0.006003,-0.005886,0.004307,0.005273,-0.002276,-0.006003,-0.004833,0,1704
1,CHUM-002,-0.006060,-0.005258,0.000122,-0.003352,0.001866,-0.000670,0.000099,-0.004938,-0.002726,...,-0.001268,-0.006060,-0.005873,0.005005,0.006572,-0.002019,-0.006060,-0.004483,1,439
2,CHUM-006,0.006663,0.003941,0.002593,0.006524,-0.001161,-0.000918,-0.005707,0.006610,0.004425,...,0.000761,0.006665,0.006503,-0.003080,-0.003952,0.001996,0.006665,0.004968,0,1186
3,CHUM-007,0.006676,0.004293,0.001961,0.006474,-0.005855,0.000587,-0.004173,0.006583,0.004402,...,0.000425,0.006676,0.006676,-0.002387,-0.001664,0.003379,0.006676,0.006145,0,1702
4,CHUM-008,-0.006672,-0.005507,-0.001959,-0.005280,0.001628,-0.000958,-0.000569,-0.005519,-0.003531,...,0.000808,-0.006672,-0.006657,0.003432,0.004565,-0.002438,-0.006672,-0.006045,0,1499


In [165]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)